MNIST with ensemble

In [1]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# Check out https://www.tensorflow.org/get_started/mnist/beginners for

# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
tf.reset_default_graph()

class Model:

    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()

    def _build_net(self):
        with tf.variable_scope(self.name):
            # dropout (keep_prob) rate  0.7~0.5 on training, but should be 1
            # for testing
            self.training = tf.placeholder(tf.bool)

            # input place holders
            self.X = tf.placeholder(tf.float32, [None, 784])
            # img 28x28x1 (black/white)
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])

            # Convolutional Layer #1
            conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3, 3],
                                     padding="SAME", activation=tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                                            padding="SAME", strides=2)
            dropout1 = tf.layers.dropout(inputs=pool1,
                                         rate=0.7, training=self.training)

            # Convolutional Layer #2
            conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3, 3],
                                     padding="SAME", activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],
                                            padding="SAME", strides=2)
            dropout2 = tf.layers.dropout(inputs=pool2,
                                         rate=0.7, training=self.training)

            # L4 FC 7x7x64 inputs -> 1/8 outputs
            flat = tf.reshape(dropout2, [-1, 7 * 7 * 64])
            dense4 = tf.layers.dense(inputs=flat,
                                     units=392, activation=tf.nn.relu)

            # L5 Final FC 625 inputs -> 10 outputs
            self.logits = tf.layers.dense(inputs=dense4, units=10)

        # define cost/loss & optimizer
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=self.logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate=learning_rate).minimize(self.cost)

        is_correct = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

    def predict(self, x_test, training=False):
        return self.sess.run(self.logits, feed_dict={self.X: x_test, self.training: training})

    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy, feed_dict={
            self.X: x_test, self.Y: y_test, self.training: training})

    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={
            self.X: x_data, self.Y: y_data, self.training: training})

In [3]:
# initialize
sess = tf.Session()

model = []
model_num = 8
for i in range(model_num):
    model.append(Model(sess, "m%d" % (i)))

sess.run(tf.global_variables_initializer())

print('Learning Started!')

# train my model
for epoch in range(training_epochs):
    avg_cost = np.zeros(len(model))
    batch_num = int(mnist.train.num_examples / batch_size)

    for i in range(batch_num):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        for j,  m in enumerate(model):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost[j] += c / batch_num

    print('Epoch:', '%04d' % (epoch + 1), 'cost =',  avg_cost)

print('Learning Finished!')

# Test model and check accuracy
batch_num = int(mnist.test.num_examples / batch_size)
avg_accuracy = np.zeros(len(model))
for i in range(batch_num):
    batch_xs, batch_ys = mnist.test.next_batch(batch_size)
    for j, m in enumerate(model):
        avg_accuracy[j] += m.get_accuracy(batch_xs, batch_ys)

for j in range(len(model)):
    avg_accuracy[j] /= batch_num
print('Accuracy:', avg_accuracy)

Learning Started!
Epoch: 0001 cost = [0.41251313 0.42328978 0.41225062 0.41497503 0.42222453 0.40887272
 0.41130106 0.42985639]
Epoch: 0002 cost = [0.16120929 0.16483014 0.16629498 0.16605567 0.16731532 0.16624435
 0.16247769 0.17093551]
Epoch: 0003 cost = [0.12307712 0.12568915 0.13040593 0.12912624 0.12624494 0.12429927
 0.12278503 0.13319425]
Epoch: 0004 cost = [0.10553371 0.10863898 0.10854822 0.10641053 0.10502257 0.10658607
 0.10107995 0.11139907]
Epoch: 0005 cost = [0.09232106 0.09548053 0.09632239 0.09519024 0.09277747 0.09687999
 0.09139883 0.09655672]
Epoch: 0006 cost = [0.08673176 0.08579051 0.08786664 0.08817582 0.0857495  0.08548708
 0.08557884 0.08992492]
Epoch: 0007 cost = [0.07813509 0.07972264 0.07948864 0.08192495 0.08196758 0.0809626
 0.07847982 0.07951118]
Epoch: 0008 cost = [0.07413214 0.07399826 0.07839298 0.07699236 0.0752342  0.07788755
 0.07615733 0.0790572 ]
Epoch: 0009 cost = [0.07128346 0.07272536 0.07053555 0.0724069  0.07026306 0.07205596
 0.07003879 0.072

In [4]:
# Test model
test_size = len(mnist.test.labels)
predictions = np.zeros(test_size * 10).reshape(test_size, 10)
for j, m in enumerate(model):
    p = m.predict(mnist.test.images)
    predictions += p

ensemble_correct_prediction = tf.equal(
    tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))
ensemble_accuracy = tf.reduce_mean(
    tf.cast(ensemble_correct_prediction, tf.float32))
print("Ensemble Accuracy:", sess.run(ensemble_accuracy))

Ensemble Accuracy: 0.9934
